<a href="https://colab.research.google.com/github/bryanbayup/Machine-Learning/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim keras-tuner imbalanced-learn Sastrawi sentencepiece seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 857.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 12.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=71c706ad119ac9c2c97266e4a15c8b43d058a8d67e97789fd6e698b5c4847ebc
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
# Download FastText
!wget -O id.tar.gz "https://www.dropbox.com/scl/fi/sju4o3keikox69euw51vy/id.tar.gz?rlkey=5jr3ijtbdwfahq7xcgig28qvy&e=1&st=gntzkzeo&dl=1"
!tar -xzf id.tar.gz

--2024-12-06 09:01:01--  https://www.dropbox.com/scl/fi/sju4o3keikox69euw51vy/id.tar.gz?rlkey=5jr3ijtbdwfahq7xcgig28qvy&e=1&st=gntzkzeo&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7e43568fd796229ffc0a4f29d9.dl.dropboxusercontent.com/cd/0/inline/CfsSAnFxfMRmJDQPTYBhu9zlAAV5b5WrkiGbXaVzK7Q7-89aWp4hlxEdvWwjgpIMQQg_3Mf_AM8AlD-UPuwjyCPeI9D5Ky0WvwALlimffXB9Hrn9U1ozZCebcNxM4zqedX0/file?dl=1# [following]
--2024-12-06 09:01:02--  https://uc7e43568fd796229ffc0a4f29d9.dl.dropboxusercontent.com/cd/0/inline/CfsSAnFxfMRmJDQPTYBhu9zlAAV5b5WrkiGbXaVzK7Q7-89aWp4hlxEdvWwjgpIMQQg_3Mf_AM8AlD-UPuwjyCPeI9D5Ky0WvwALlimffXB9Hrn9U1ozZCebcNxM4zqedX0/file?dl=1
Resolving uc7e43568fd796229ffc0a4f29d9.dl.dropboxusercontent.com (uc7e43568fd796229ffc0a4f29d9.dl.dropboxusercontent.com)... 162.125.2.15, 2620:100:6017:15::a2

In [8]:
# --------------------------------------
# Bagian 1: Instalasi dan Import Library
# --------------------------------------
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, Dropout, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import re
import pickle
import os
import random
import nltk
import gensim
from gensim.models import KeyedVectors
from keras_tuner import HyperModel, RandomSearch
from tensorflow.keras.callbacks import EarlyStopping
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from seqeval.metrics import classification_report as seq_classification_report

nltk.download('stopwords')

# --------------------------------------
# Bagian 2: Memuat FastText
# --------------------------------------
try:
    fasttext_model = KeyedVectors.load_word2vec_format('id.vec', binary=False)
    print("FastText 'id.vec' berhasil dimuat.")
except Exception as e:
    print(f"Gagal memuat 'id.vec': {e}")
    raise ValueError("Gagal memuat FastText.")

# --------------------------------------
# Bagian 3: Memuat dataset dari data.json
# Pastikan 'data.json' adalah file JSON yang berisi array percakapan seperti yang Anda berikan.
# --------------------------------------
with open('data2.json', 'r', encoding='utf-8') as f:
    conversations = json.load(f)

# Data format:
# [
#   {
#       "conversation_id": "...",
#       "turns": [
#          {
#             "speaker": "user",
#             "utterance": "...",
#             "entities": [...],
#             "intent": "..."
#          },
#          {
#             "speaker": "bot",
#             "utterance": "...",
#             "entities": [...],
#             "intent": "..."
#          },
#          ...
#       ]
#   },
#   ...
# ]

# Kita ekstrak hanya turn user, karena kita ingin melatih model intent & NER dari perspektif user input.
user_utterances = []
intents = []
entity_labels = []

def char_offset_to_token_labels(utterance, entities, tokenizer=lambda x: x.split()):
    tokens = tokenizer(utterance)
    labels = ["O"] * len(tokens)
    # buat mapping token ke character ranges
    char_pos = 0
    token_ranges = []
    for t in tokens:
        start_pos = char_pos
        end_pos = start_pos + len(t)
        token_ranges.append((start_pos, end_pos))
        char_pos = end_pos + 1  # spasi

    for ent in entities:
        ent_start = ent['start']
        ent_end = ent['end']
        ent_type = ent['entity'].upper()
        ent_token_positions = []
        for i, (ts, te) in enumerate(token_ranges):
            if not (te <= ent_start or ts >= ent_end):
                ent_token_positions.append(i)
        if len(ent_token_positions) > 0:
            labels[ent_token_positions[0]] = "B-" + ent_type
            for p in ent_token_positions[1:]:
                labels[p] = "I-" + ent_type
    return tokens, labels

for conv in conversations:
    for turn in conv["turns"]:
        if turn["speaker"] == "user":
            utt = turn["utterance"]
            ents = turn.get("entities", [])
            intent = turn.get("intent", "None")
            tokens, ner_tags = char_offset_to_token_labels(utt, ents)
            user_utterances.append(tokens)
            intents.append(intent)
            entity_labels.append(ner_tags)

# --------------------------------------
# Bagian 4: Preprocessing (Cleaning, Stopwords, Stemming)
# --------------------------------------
with open('stopword_list_tala.txt', 'r', encoding='utf-8') as f:
    stop_words = f.read().splitlines()
stop_words = set(word.strip().lower() for word in stop_words)

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

def preprocess_text(text):
    text = clean_text(text)
    tokens = text.split()
    tokens = [w for w in tokens if w not in stop_words]
    tokens = [stemmer.stem(w) for w in tokens]
    return ' '.join(tokens)

# Kita akan memproses user_utterances menjadi kalimat lagi untuk TF-IDF nanti
utterances_joined = [' '.join(utt) for utt in user_utterances]
utterances_clean = [preprocess_text(u) for u in utterances_joined]

# --------------------------------------
# Bagian 5: Buat DataFrame untuk balancing
# Kita perlu intent untuk balancing. Kita akan gunakan oversampling sesuai intent.
# --------------------------------------
df_data = pd.DataFrame({
    'utterances': utterances_joined,
    'intent': intents,
    'entities': entity_labels,
    'utterances_clean': utterances_clean
})

label_encoder = LabelEncoder()
df_data['intent_label'] = label_encoder.fit_transform(df_data['intent'])
intent_mapping = dict(zip(df_data['intent_label'], df_data['intent']))

# Handling imbalance
ros = RandomOverSampler(random_state=42)
X = df_data.index.values.reshape(-1, 1)
y = df_data['intent_label']
X_ros, y_ros = ros.fit_resample(X, y)
df_balanced = df_data.loc[X_ros.flatten()].reset_index(drop=True)
df_balanced['intent_label'] = y_ros
df_balanced['intent'] = label_encoder.inverse_transform(df_balanced['intent_label'])

# --------------------------------------
# Bagian 6: Persiapan data untuk Intent Classification
# --------------------------------------
texts = df_balanced['utterances_clean'].tolist()
labels = df_balanced['intent_label'].tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='')
tokenizer.fit_on_texts(train_texts)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)
max_seq_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in val_sequences))
train_padded = pad_sequences(train_sequences, maxlen=max_seq_length, padding='post')
val_padded = pad_sequences(val_sequences, maxlen=max_seq_length, padding='post')

num_classes = len(label_encoder.classes_)
train_labels_cat = to_categorical(train_labels, num_classes=num_classes)
val_labels_cat = to_categorical(val_labels, num_classes=num_classes)

# Buat embedding matrix
embedding_dim = fasttext_model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in word_index.items():
    if word in fasttext_model:
        embedding_matrix[idx] = fasttext_model[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

# --------------------------------------
# Bagian 7: Persiapan data NER
# Kita harus re-build texts_ner, labels_ner dari df_balanced (yang sudah balanced).
# Namun balancing mungkin mengulang sample, hal ini tidak masalah untuk contoh ini.
# Yang penting data NER diambil dari df_balanced (karena df_balanced adalah data final).
# --------------------------------------
# NER label set
all_labels = set()
for tags in df_balanced['entities']:
    for t in tags:
        if t != 'O':
            all_labels.add(t)
all_labels.add('O')
all_labels = sorted(list(all_labels))
ner_label_encoder = {label: idx for idx, label in enumerate(all_labels)}
ner_label_decoder = {idx: label for label, idx in ner_label_encoder.items()}

# Encode tags function
def encode_tags(tags, max_len):
    tag_ids = [ner_label_encoder[t] for t in tags]
    tag_ids = tag_ids[:max_len] + [ner_label_encoder['O']]*(max_len - len(tag_ids))
    return tag_ids

# Kita perlu text sequence untuk NER dari df_balanced
# Token ulang berdasarkan tokenizer yang sama
def text_to_sequence(text):
    # text disini sudah berbentuk token hasil join. Kita gunakan tokenizer.
    seq = tokenizer.texts_to_sequences([text])
    return seq[0]

X_ner = []
Y_ner = []
for i, row in df_balanced.iterrows():
    seq = text_to_sequence(row['utterances_clean'])
    seq_padded = seq[:max_seq_length] + [0]*(max_seq_length - len(seq))
    X_ner.append(seq_padded)
    tag_ids = encode_tags(row['entities'], max_seq_length)
    Y_ner.append(tag_ids)

X_ner = np.array(X_ner)
Y_ner = np.array(Y_ner)
Y_ner = to_categorical(Y_ner, num_classes=len(ner_label_encoder))

# split data untuk NER
train_texts_ner, val_texts_ner, train_labels_ner, val_labels_ner = train_test_split(
    X_ner,
    Y_ner,
    test_size=0.2,
    random_state=42
)

# --------------------------------------
# Bagian 8: Membangun Model Intent (CNN)
# --------------------------------------
def build_intent_model_with_cnn(embedding_matrix, max_seq_length, num_classes, l2_reg=0.001):
    inputs = Input(shape=(max_seq_length,))
    embedding = tf.keras.layers.Embedding(
        input_dim=embedding_matrix.shape[0],
        output_dim=embedding_matrix.shape[1],
        weights=[embedding_matrix],
        trainable=True
    )(inputs)
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding)
    global_pool = GlobalMaxPooling1D()(conv)
    dense = Dense(64, activation='relu', kernel_regularizer=l2(l2_reg))(global_pool)
    dropout = Dropout(0.5)(dense)
    outputs = Dense(num_classes, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=outputs)
    return model

model_intent_cnn = build_intent_model_with_cnn(embedding_matrix, max_seq_length, num_classes, l2_reg=0.001)
model_intent_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks_intent = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
]

history_intent_cnn = model_intent_cnn.fit(
    train_padded,
    train_labels_cat,
    validation_data=(val_padded, val_labels_cat),
    epochs=20,
    batch_size=16,
    callbacks=callbacks_intent
)

# --------------------------------------
# Bagian 9: Hyperparameter Tuning untuk NER
# --------------------------------------
class NERHyperModel(HyperModel):
    def __init__(self, embedding_matrix, max_seq_length, num_entities):
        self.embedding_matrix = embedding_matrix
        self.max_seq_length = max_seq_length
        self.num_entities = num_entities

    def build(self, hp):
        l2_reg = hp.Choice('l2_reg', values=[1e-4, 1e-3, 1e-2])
        dropout_rate = hp.Float('dropout_rate', 0.3, 0.7, step=0.1)
        lstm_units = hp.Int('lstm_units', min_value=32, max_value=128, step=32)

        inputs = Input(shape=(self.max_seq_length,))
        embedding = tf.keras.layers.Embedding(
            input_dim=self.embedding_matrix.shape[0],
            output_dim=self.embedding_matrix.shape[1],
            weights=[self.embedding_matrix],
            trainable=True
        )(inputs)
        lstm = Bidirectional(LSTM(lstm_units, kernel_regularizer=l2(l2_reg), return_sequences=True))(embedding)
        dropout = Dropout(dropout_rate)(lstm)
        outputs = TimeDistributed(Dense(self.num_entities, activation='softmax'))(dropout)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

ner_hypermodel = NERHyperModel(embedding_matrix, max_seq_length, len(ner_label_encoder))

tuner_ner = RandomSearch(
    ner_hypermodel,
    objective='val_accuracy',
    max_trials=10,  # kurangi trials untuk cepat
    executions_per_trial=1,
    directory='ner_tuner_dir',
    project_name='ner_tuning'
)

tuner_ner.search(
    train_texts_ner,
    train_labels_ner,
    epochs=10,
    validation_data=(val_texts_ner, val_labels_ner),
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    ]
)

best_model_ner = tuner_ner.get_best_models(num_models=1)[0]
best_hp_ner = tuner_ner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters for NER: {best_hp_ner.values}")

loss_intent, accuracy_intent = model_intent_cnn.evaluate(val_padded, val_labels_cat)
print(f'Akurasi Model Klasifikasi Intent: {accuracy_intent * 100:.2f}%')

loss_ner, accuracy_ner = best_model_ner.evaluate(val_texts_ner, val_labels_ner)
print(f'Akurasi Model NER: {accuracy_ner * 100:.2f}%')

# --------------------------------------
# Bagian 10: Simpan Model dan Encoder
# --------------------------------------
os.makedirs('models', exist_ok=True)
os.makedirs('encoders', exist_ok=True)
os.makedirs('data', exist_ok=True)

model_intent_cnn.save('models/model_intent.keras')
best_model_ner.save('models/model_ner.keras')

with open('encoders/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('encoders/label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('encoders/ner_label_encoder.pickle', 'wb') as handle:
    pickle.dump(ner_label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Buat DataFrame untuk TF-IDF responses
# Karena dataset ini tidak punya field "responses" dari user,
# kita hanya gunakan utterances sebagai contoh. Jika butuh response, siapkan dataset serupa.
# Di dataset ini, sebagian besar intent adalah "Mendiagnosis Gejala" atau "Rekomendasi Penanganan Awal".
# Tidak ada explicit 'responses' field dari user. Asumsi: Gunakan utterances sendiri sebagai basis retrieval.
df_utterances = df_balanced[['utterances', 'intent', 'utterances_clean']].reset_index(drop=True)

vectorizer = TfidfVectorizer()
vectorizer.fit(df_utterances['utterances_clean'])

with open('data/vectorizer.pickle', 'wb') as handle:
    pickle.dump(vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# --------------------------------------
# Bagian 11: intent_animal_mapping
# (Dari informasi yang Anda berikan)
# --------------------------------------
intent_animal_mapping = {
    "Melaporkan Hewan Terlantar": ["kucing", "anjing"],
    "Mendiagnosis Gejala": ["kucing", "anjing"],
    "Rekomendasi Penanganan Awal": ["kucing", "anjing"],
    "Konfirmasi Laporan": ["kucing", "anjing"],
    "Tindak Lanjut Laporan": ["kucing", "anjing"],
    "Rekomendasi Tindakan": ["kucing", "anjing"]
}

# --------------------------------------
# Bagian 12: Fungsi Inference
# --------------------------------------
def predict_intent(text):
    text_clean = preprocess_text(text)
    seq = tokenizer.texts_to_sequences([text_clean])
    padded_seq = pad_sequences(seq, maxlen=max_seq_length, padding='post')
    pred = model_intent_cnn.predict(padded_seq)
    predicted_label = np.argmax(pred, axis=1)[0]
    intent = label_encoder.inverse_transform([predicted_label])[0]
    return intent

def predict_entities(text):
    text_clean = preprocess_text(text)
    seq = tokenizer.texts_to_sequences([text_clean])
    padded_seq = pad_sequences(seq, maxlen=max_seq_length, padding='post')
    pred = best_model_ner.predict(padded_seq)
    pred_labels = np.argmax(pred, axis=-1)[0]
    tokens = tokenizer.sequences_to_texts(seq)[0].split()
    entities = []
    for idx, label_id in enumerate(pred_labels[:len(tokens)]):
        label = ner_label_decoder[label_id]
        if label != 'O':
            entities.append({'entity': label.split('-')[1].lower(), 'value': tokens[idx]})
    return entities

def adjust_intent(intent, entities):
    predicted_animals = intent_animal_mapping.get(intent, None)
    entity_animals = [ent['value'].lower() for ent in entities if ent['entity'] == 'animal']
    if entity_animals:
        user_animal = entity_animals[0]
        if predicted_animals and user_animal not in predicted_animals:
            # Coba cari intent lain yang cocok dgn animal ini
            for i_name, animals in intent_animal_mapping.items():
                if user_animal in animals:
                    intent = i_name
                    break
            else:
                intent = None
    else:
        # Jika intent butuh hewan tapi tidak ditemukan, intent bisa diset None atau dibiarkan
        # Disini kita biarkan saja
        pass
    return intent

def get_default_response():
    default_responses = [
        "Maaf, saya belum bisa menjawab pertanyaan Anda.",
        "Maaf, mohon diperjelas apa yang Anda maksud.",
        "Maaf, saya hanya diprogram untuk menjawab pertanyaan mengenai kucing dan anjing.",
        "Mohon maaf, saya tidak mengerti. Bisa dijelaskan lebih detail?",
        "Saya belum memiliki informasi mengenai hal tersebut."
    ]
    return random.choice(default_responses)

def get_response(user_input, intent=None, entities=None):
    user_input_clean = preprocess_text(user_input)
    if intent:
        df_intent = df_utterances[df_utterances['intent'] == intent]
        if df_intent.empty:
            return get_default_response()
        else:
            tfidf_matrix_intent = vectorizer.transform(df_intent['utterances_clean'])
            user_tfidf = vectorizer.transform([user_input_clean])
            similarities = cosine_similarity(user_tfidf, tfidf_matrix_intent)
            most_similar_idx = np.argmax(similarities[0])
            highest_similarity = similarities[0][most_similar_idx]
            if highest_similarity < 0.2:
                return get_default_response()
            else:
                # Karena dataset ini tidak menyediakan field responses khusus (selain user),
                # Anda perlu menyesuaikan. Disini kita akan hanya mengembalikan utterance itu sendiri
                # sebagai contoh, atau gunakan get_default_response().
                return "Saya menyarankan untuk konsultasi lebih lanjut ke dokter hewan."
    else:
        return get_default_response()

# --------------------------------------
# Bagian 13: Multi-Turn Handling
# --------------------------------------
dialog_history = []
MAX_HISTORY = 2

def predict_intent_multi_turn(dialog_history, current_input):
    history_context = dialog_history[-MAX_HISTORY:] if len(dialog_history) > MAX_HISTORY else dialog_history
    combined_input = " ".join(history_context + [current_input])
    text_clean = preprocess_text(combined_input)
    seq = tokenizer.texts_to_sequences([text_clean])
    padded_seq = pad_sequences(seq, maxlen=max_seq_length, padding='post')
    pred = model_intent_cnn.predict(padded_seq)
    predicted_label = np.argmax(pred, axis=1)[0]
    intent = label_encoder.inverse_transform([predicted_label])[0]
    return intent

def predict_entities_multi_turn(dialog_history, current_input):
    history_context = dialog_history[-MAX_HISTORY:] if len(dialog_history) > MAX_HISTORY else dialog_history
    combined_input = " ".join(history_context + [current_input])
    text_clean = preprocess_text(combined_input)
    seq = tokenizer.texts_to_sequences([text_clean])
    padded_seq = pad_sequences(seq, maxlen=max_seq_length, padding='post')
    pred = best_model_ner.predict(padded_seq)
    pred_labels = np.argmax(pred, axis=-1)[0]
    tokens = tokenizer.sequences_to_texts(seq)[0].split()
    entities = []
    for idx, label_id in enumerate(pred_labels[:len(tokens)]):
        label = ner_label_decoder[label_id]
        if label != 'O':
            entities.append({'entity': label.split('-')[1].lower(), 'value': tokens[idx]})
    return entities

def chatbot_response_multi_turn(user_input):
    dialog_history.append(user_input)
    intent = predict_intent_multi_turn(dialog_history, user_input)
    entities = predict_entities_multi_turn(dialog_history, user_input)
    adjusted_intent = adjust_intent(intent, entities)

    if adjusted_intent is None:
        response = get_default_response()
    else:
        response = get_response(user_input, adjusted_intent, entities)
        if not response:
            response = get_default_response()
    return response

# --------------------------------------
# Bagian 14: Contoh Pengujian Multi-turn
# --------------------------------------
print("=== Simulasi Multi-Turn ===")
input_turns = [
    "Halo, saya punya pertanyaan tentang anjing saya.",
    "Anjing saya selalu menggaruk telinganya.",
    "Dia juga ada cairan keluar dari telinganya, apa yang harus saya lakukan?"
]

for user_text in input_turns:
    print(f"Anda: {user_text}")
    resp = chatbot_response_multi_turn(user_text)
    print(f"Chatbot: {resp}\n")

Trial 10 Complete [00h 00m 13s]
val_accuracy: 0.8206349611282349

Best val_accuracy So Far: 0.8301587104797363
Total elapsed time: 00h 06m 42s
Best Hyperparameters for NER: {'l2_reg': 0.0001, 'dropout_rate': 0.4, 'lstm_units': 96}
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9246 - loss: 0.2354 


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Akurasi Model Klasifikasi Intent: 94.29%
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8196 - loss: 0.5805 
Akurasi Model NER: 83.02%
=== Simulasi Multi-Turn ===
Anda: Halo, saya punya pertanyaan tentang anjing saya.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step
Chatbot: Saya menyarankan untuk konsultasi lebih lanjut ke dokter hewan.

Anda: Anjing saya selalu menggaruk telinganya.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Chatbot: Saya menyarankan untuk konsultasi lebih lanjut ke dokter hewan.

Anda: Dia juga ada cairan keluar dari telinganya, apa yang harus saya lakukan?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Chatbot: Saya menyarankan untuk konsultasi lebih lanjut ke dokter hewan.



In [9]:
print("=== Simulasi Multi-Turn ===")
input_turns = [
    "Anjing saya tidak nafsu makan dan sering diare. Apa yang terjadi?",
    "Tidak, hanya diare saja.",
    "Terima kasih"
]

for user_text in input_turns:
    print(f"Anda: {user_text}")
    resp = chatbot_response_multi_turn(user_text)
    print(f"Chatbot: {resp}\n")

=== Simulasi Multi-Turn ===
Anda: Anjing saya tidak nafsu makan dan sering diare. Apa yang terjadi?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Chatbot: Maaf, saya hanya diprogram untuk menjawab pertanyaan mengenai kucing dan anjing.

Anda: Tidak, hanya diare saja.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Chatbot: Saya menyarankan untuk konsultasi lebih lanjut ke dokter hewan.

Anda: Terima kasih
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Chatbot: Saya menyarankan untuk konsultasi lebih lanjut ke dokter hewan.

